<a href="https://colab.research.google.com/github/BrotherKim/KAIST_CODE/blob/master/SEP531/term/4_KoBERT_emotion_finetune_jr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#변수 정의

In [32]:
ROOT_DIR = '/content/drive/MyDrive/KAIST/SEP531/kor_jr_labeled'
TRAIN_DIR = '%s/train' % ROOT_DIR
VALID_DIR = '%s/valid' % ROOT_DIR

NUM_TRAIN = 171
NUM_VALID = 58

SAVEPOINT_PATH = '/content/drive/MyDrive/KAIST/SEP531/KoBERT_emotion_finetuned_data_jr.pt'

#실행 환경#

- Python >= 3.6
- PyTorch >= 1.70
- Transformers = 3.0.2
- Colab
- batch size = 64 (convertable)
- epochs = 10 (convertable)

In [33]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

  Using cached transformers-3.0.2-py3-none-any.whl (769 kB)
  Using cached tokenizers-0.8.1rc1-cp37-cp37m-manylinux1_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.10.3
    Uninstalling tokenizers-0.10.3:
      Successfully uninstalled tokenizers-0.10.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.13.0
    Uninstalling transformers-4.13.0:
      Successfully uninstalled transformers-4.13.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kobert 0.1.2 requires transformers>=4.8.1, but you have transformers 3.0.2 which is incompatible.


#KoBERT 다운로드#

In [34]:
#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-3q4qs1n9
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-3q4qs1n9
  Using cached transformers-4.13.0-py3-none-any.whl (3.3 MB)
  Using cached tokenizers-0.10.3-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (3.3 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.8.1rc1
    Uninstalling tokenizers-0.8.1rc1:
      Successfully uninstalled tokenizers-0.8.1rc1
  Attempting uninstall: transformers
    Found existing installation: transformers 3.0.2
    Uninstalling transformers-3.0.2:
      Successfully uninstalled transformers-3.0.2


In [35]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [36]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [37]:
#GPU 사용
device = torch.device("cuda:0")

In [38]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


#데이터셋 전처리#

In [39]:
#구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
# 터미널 커맨드를 문자열로 반환해주는 함수 작성

import subprocess

def GetShellCmdStdOut(command):
  cmd = ['sh', '-c', command]
  fd_popen = subprocess.Popen(cmd, stdout=subprocess.PIPE).stdout 
  data = fd_popen.read().strip() 
  fd_popen.close()

  retval = data.decode('utf-8') 
  return retval

In [41]:
%pushd $TRAIN_DIR
tl = GetShellCmdStdOut('find | grep answer').split('\n')
train_json_list = [x.replace('./', '') for x in tl]
train_json_list.sort()
%popd

/content/drive/MyDrive/KAIST/SEP531/kor_jr_labeled/train
/content
popd -> /content


In [42]:
%pushd $VALID_DIR
vl = GetShellCmdStdOut('find | grep answer').split('\n')
valid_json_list = [x.replace('./', '') for x in vl]
valid_json_list.sort()
%popd

/content/drive/MyDrive/KAIST/SEP531/kor_jr_labeled/valid
/content
popd -> /content


In [43]:
train_json_list[0]

'DJSX20003906.json.answer'

In [44]:
!cat $TRAIN_DIR/{train_json_list[0]}

언니 근데 언니 올해 들어서 되게 머리 스타일에 관심이 많아진 거 같애.	언니 근데 언니 올해 들어서 되게 머리 스타일에 관심이 많아진 거 같애.	3
어 나 왜 그랬냐면은 올해 우리 아가씨야 우리 아가씨가 그러는 거야.	어 나 왜 그랬냐면은 올해 우리 아가씨야 우리 아가씨가 그러는 거야.	3
나한테 언니 언니 머리 너무 까만 색 아니야?	나한테 언니 언니 머리 너무 까만 색 아니야?	0
이렇게 하는 거야 그래가지고 내가	이렇게 하는 거야 그래가지고 내가	3
왜요 이렇게 했더니 언니 여름에는 그렇게 까만색 하면 더워보여 이렇게 하는거야.	왜요 이렇게 했더니 언니 여름에는 그렇게 까만색 하면 더워보여 이렇게 하는거야.	2
그래서 내가	그래서 내가	4
너무 짜증이 나는 거지 나보다 못생긴 애한테 그런 소리 들으니까 그래서 내가	너무 짜증이 나는 거지 나보다 못생긴 애한테 그런 소리 들으니까 그래서 내가	6
아차 싶어가지고 언니랑 같이 미용실을 가기 시작했어.	아차 싶어가지고 언니랑 같이 미용실을 가기 시작했어.	3
요즘 언니 머리 색깔을 보면은 무지개 후려쳐.	요즘 언니 머리 색깔을 보면은 무지개 후려쳐.	6
그리고 나도 옛날에 왕년에 염색 조금 했었는데 요즘 언니 머리 색깔이 더 눈에 띄고	글고 나도 옛날에 왕년에 염색 쫌 했었는데 요즘 언니 머리 색깔이 더 눈에 띄고	4
뭔 고딩(같어.)/(같아.)	뭔 고딩(같어.)/(같아.)	3
아니 내가 이 머리 색깔 할려고 한 게 아니야 내 머리 색깔이 원래 애쉬 애쉬 애쉬 애쉬그레이라고 있어.	아니 내가 이 머리 색깔 할려고 한 게 아니야 내 머리 색깔이 원래 애쉬 애쉬 애쉬 애쉬그레이라고 있어.	4
애쉬 그레이라고 애쉬 그레이라고 있는데	애쉬 그레이라고 애쉬 그레이라고 있는데	4
이 애쉬그레이는 탈색을 열 두 번은 넘게 해야지 색깔이 나온다는 거야.	이 애쉬그레이는 탈색을 열 두 번은 넘게 해야지 색깔이 나온다는 거야.	4
그러면은	그러면은	0
에쉬 그레이 가 아니라 에이씨 그레이네.	에쉬 그레이 가 아니라 에이씨 그레

In [45]:
import pandas as pd

chatbot_data = pd.read_csv('%s/%s' % (TRAIN_DIR, train_json_list[0]), sep='\t', header=None)

chatbot_data

,0,1,2
0,언니 근데 언니 올해 들어서 되게 머리 스타일에 관심이 많아진 거 같애.,언니 근데 언니 올해 들어서 되게 머리 스타일에 관심이 많아진 거 같애.,3
1,어 나 왜 그랬냐면은 올해 우리 아가씨야 우리 아가씨가 그러는 거야.,어 나 왜 그랬냐면은 올해 우리 아가씨야 우리 아가씨가 그러는 거야.,3
2,나한테 언니 언니 머리 너무 까만 색 아니야?,나한테 언니 언니 머리 너무 까만 색 아니야?,0
3,이렇게 하는 거야 그래가지고 내가,이렇게 하는 거야 그래가지고 내가,3
4,왜요 이렇게 했더니 언니 여름에는 그렇게 까만색 하면 더워보여 이렇게 하는거야.,왜요 이렇게 했더니 언니 여름에는 그렇게 까만색 하면 더워보여 이렇게 하는거야.,2
...,...,...,...
226,안 할 수밖에 없는 거고 그러니까,안 할 수밖에 없는 거고 그니까,3
227,본인이 우리가 메이크업 아티스트가 아닌 이상,본인이 우리가 메이크업 아티스트가 아닌 이상,0
228,모든 사람의 얼굴을 다 해보지는 않잖아 어떤 사람이 이걸 떡 척보면 이게 잘 어울리...,모든 사람의 얼굴을 다 해보지는 않잖아 어떤 사람이 이걸 떡 척보면 이게 잘 어울리...,4
229,그러니까 그냥 자기가 하는 방법대로 남한테 해주면 안 어울리지 당연히 안 어울리지.,그러니까 그냥 자기가 하는 방법대로 남한테 해주면 안 어울리지 당연히 안 어울리지.,6


In [46]:
def GenDataset(DIR, json_list):
  retval = []

  for fn in json_list:
    #print('%s\n' % (fn))
    fd = pd.read_csv('%s/%s' % (DIR, fn), sep='\t', header=None)
    # [1]데이터가 사투리, [2]가 감정
    for q, label in zip(fd[1], fd[2])  :
      #print('%s, %s\n' % (q, label))
      data = []
      data.append(q)
      data.append(str(label))
      retval.append(data)

  return retval

In [47]:
dataset_train = GenDataset(TRAIN_DIR, train_json_list)

In [48]:
dataset_valid = GenDataset(VALID_DIR, valid_json_list)

In [49]:
print(len(dataset_train))
print(len(dataset_valid))

57301
26495


In [50]:
dataset_train = dataset_train[0:45000]
dataset_valid = dataset_valid[0:15000]

In [51]:
print(len(dataset_train))
print(len(dataset_valid))

45000
15000


#KoBERT 입력 데이터로 변환#

In [52]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [53]:
# Setting parameters
max_len = 64
batch_size = 64 # origin : 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [54]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_valid = BERTDataset(dataset_valid, 0, 1, tok, max_len, True, False)

using cached model


In [55]:
# 첫번째는 패딩된 시퀀스, 두번째는 길이와 티입, 세번째는 어텐션 마스크 시퀀스
data_train[0]

(array([   2, 3241, 5770, 1221, 5850, 3241, 5770, 3450, 1809, 1763, 5400,
        2008, 2939, 6896, 1091, 1952, 7344,  862,  830, 6834,  517,   54,
           3,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(23, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 3)

In [56]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_valid, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


#KoBERT 학습모델 만들기#

In [57]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [58]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)
model.load_state_dict(torch.load('/content/drive/MyDrive/KAIST/SEP531/KoBERT_emotion_finetuned_data_added.pt'), strict=False)
model.eval()

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    

#KoBERT 모델 학습시키기#

In [60]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/704 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 0.7136996388435364 train acc 0.765625
epoch 1 batch id 201 loss 1.0362825393676758 train acc 0.7318874378109452
epoch 1 batch id 401 loss 0.6966012120246887 train acc 0.7116583541147132
epoch 1 batch id 601 loss 0.5756672620773315 train acc 0.7052828618968386
epoch 1 train acc 0.705322265625


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 4 test acc 0.7619902482269504


  0%|          | 0/704 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.05144962668418884 train acc 1.0
epoch 5 batch id 201 loss 0.18423576653003693 train acc 0.9615205223880597
epoch 5 batch id 401 loss 0.10283415019512177 train acc 0.9649703865336658
epoch 5 batch id 601 loss 0.06712385267019272 train acc 0.9676060732113144
epoch 5 train acc 0.9685724431818182


  0%|          | 0/235 [00:00<?, ?it/s]

epoch 5 test acc 0.7636746453900709


In [61]:
torch.save(model.state_dict(), SAVEPOINT_PATH)